Utilizando deap

In [2]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.2 MB/s eta 0:00:00


In [4]:
import random
from deap import base, creator, tools

# Definir el problema
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Función de evaluación
def evaluate(individual):
    x = int(''.join(map(str, individual)), 2)  # Convertir binario a decimal
    return (x**(2*x) - 1,)

# Configuración del algoritmo
toolbox = base.Toolbox()
toolbox.register("binary", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binary, n=5)  # Longitud 5
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)  # Probabilidad por bit
toolbox.register("select", tools.selTournament, tournsize=2)

# Algoritmo genético
def main():
    population = toolbox.population(n=10)  # Población de 10
    for gen in range(3):  # 3 generaciones
        for ind in population:
            ind.fitness.values = toolbox.evaluate(ind)

        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:  # Tasa de aplicación de la mutación
                toolbox.mutate(mutant)
                del mutant.fitness.values

        population[:] = offspring

    best_ind = tools.selBest(population, 1)[0]
    print("Best individual:", best_ind)
    print("Best fitness:", best_ind.fitness.values)

if __name__ == "__main__":
    main()


Best individual: [1, 1, 1, 1, 1]
Best fitness: (2.913567055049513e+92,)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Sin Deap

In [5]:
import random

# Función de evaluación
def evaluate(individual):
    x = int(''.join(map(str, individual)), 2)  # Convertir binario a decimal
    return x**(2*x) - 1

# Inicializar población
def initialize_population(size, length):
    return [[random.randint(0, 1) for _ in range(length)] for _ in range(size)]

# Selección por torneo
def tournament_selection(population):
    competitors = random.sample(population, 2)
    return max(competitors, key=evaluate)

# Cruzamiento
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    return parent1[:point] + parent2[point:]

# Mutación
def mutate(individual):
    point = random.randint(0, len(individual) - 1)
    individual[point] = 1 - individual[point]

# Algoritmo genético
def main():
    population = initialize_population(10, 5)  # Población de 10, longitud 5
    for gen in range(3):  # 3 generaciones
        new_population = []
        for _ in range(len(population)):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            if random.random() < 0.2:  # Tasa de mutación
                mutate(child)
            new_population.append(child)
        population = new_population

    best_ind = max(population, key=evaluate)
    print("Best individual:", best_ind)
    print("Best fitness:", evaluate(best_ind))

if __name__ == "__main__":
    main()

Best individual: [1, 1, 1, 0, 1]
Best fitness: 6593012181135818604873641088565793338718216041722729705516887554767627147075930273960
